<h1> GENRE: POP </h1>

<h3>(1) Access the data from the dataset </h3>


In [2]:
%%capture
%pip install datasets

from datasets import load_dataset

ds = load_dataset("sebastiandizon/genius-song-lyrics")

In [3]:
import heapq
from datasets import Dataset
type_of_music= 'pop'
TOP_K = 1000
decade_heaps = {70: [], 80: [], 90: [], 00: [], 10:[], 20:[]}  # z.B. 70=70er, 0=00er etc.

def get_decade(year):
    if 1970 <= year <= 1979:
        return 70
    elif 1980 <= year <= 1989:
        return 80
    elif 1990 <= year <= 1999:
        return 90
    elif 2000 <= year <= 2009:
        return 00
    elif 2010 <= year <= 2019:
        return 10
    elif 2020 <= year <= 2029:
        return 20
    return None

i = 0  # Zähler für tie-break
for song in ds["train"]:
    if song["tag"] != type_of_music:
        continue
    elif song["language"] != "en":
        continue
    dec = get_decade(song["year"])
    if dec is None:
        continue

    views = song["views"]
    heap = decade_heaps[dec]

    # Das Tupel enthält jetzt (views, i, song).
    # Der zweite Wert i ist rein für Tie-Break (und garantiert sortierbar).
    if len(heap) < TOP_K:
        heapq.heappush(heap, (views, i, song))
    else:
        # Vergleiche nur, ob aktuelle views (oder Tie-Break i) größer sind 
        # als das Minimum im Heap (heap[0]).
        if (views, i) > (heap[0][0], heap[0][1]):
            heapq.heapreplace(heap, (views, i, song))
    i += 1

# Am Ende hat jede Dekade max. 1000 Elemente.
decade_toplists = {}
for dec, heap in decade_heaps.items():
    # Sortieren nach (views, i) absteigend.
    # Falls dir der Index egal ist, kannst du einfach nur nach views sortieren.
    sorted_heap = sorted(heap, key=lambda x: (x[0], x[1]), reverse=True)
    # x[0] = views, x[1] = tie-break index, x[2] = song
    songs_only = [item[2] for item in sorted_heap]
    decade_toplists[dec] = songs_only

# Optional: zu HF Dataset konvertieren
subdataset_70 = Dataset.from_list(decade_toplists[70])
subdataset_80 = Dataset.from_list(decade_toplists[80])
subdataset_90 = Dataset.from_list(decade_toplists[90])
subdataset_00 = Dataset.from_list(decade_toplists[00])
subdataset_10 = Dataset.from_list(decade_toplists[10])
subdataset_20 = Dataset.from_list(decade_toplists[20])

# Save to hard drive
subdataset_70.save_to_disk("subdataset_70_pop")
subdataset_80.save_to_disk("subdataset_80_pop")
subdataset_90.save_to_disk("subdataset_90_pop")
subdataset_00.save_to_disk("subdataset_00_pop")
subdataset_10.save_to_disk("subdataset_10_pop")
subdataset_20.save_to_disk("subdataset_20_pop")




Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

<h2> LDA </h2>

<h2> Remove the bad words</h2>

In [1]:
subdataset_00[1]

NameError: name 'subdataset_00' is not defined

In [21]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tushifire/ldnoobw")
path = kagglehub.dataset_download("sahib12/badwords")

#file = kagglehub.load_dataset("tushifire/ldnoobw",path)

print("Path to dataset files:", path)

swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

with open(swear_words_file, 'r') as f:
    swear_words = set(f.read().splitlines())

print(f"!!!!Loaded {len(swear_words)} swear words.")

#print(swear_words)
print("Done")


Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done


In [22]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get", "yeah", "got","verse", "chorus", "hook", "bridge", "outro", "intro"])
print("Stopwords:", stop_words)
# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
    #doc = remove_stopwords(doc.lower())
    doc = remove_repeated_lines(doc)
    
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    
    return tokens

def remove_repeated_lines(doc):
    lines = doc.split("\n")
    return "\n".join(sorted(set(lines), key=lines.index))

def sort_out_bad_words(doc,word_list, replacement=" "):
    """
    Ersetzt Wörter in einem Text, wenn sie in einer Liste vorkommen.
    
    :param text: Der Eingabetext (String).
    :param word_list: Liste der Wörter, die ersetzt werden sollen.
    :param replacement: Der Text, durch den die Wörter ersetzt werden sollen.
    :return: Der bearbeitete Text.
    """
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)
 

def process_subdatasets(subdataset):
    # Get rid of the bad words and replace it with "swear word"
    without_swear = [sort_out_bad_words(doc, swear_words ) for doc in subdataset]

    # Preprocessing für alle Dokumente anwenden
    processed_docs = [preprocess(doc) for doc in without_swear]

    if stop_words in processed_docs:
        print("FEHLER: Stopwords nicht entfernt!")
        
    # Erstellen eines Wörterbuchs
    dictionary = Dictionary(processed_docs)

    # Erstellen eines Bag-of-Words-Korpus
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    # LDA-Modell trainieren
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=5,  # Anzahl der Themen
        random_state=42,
        passes=40,  # Anzahl der Durchgänge
        alpha='symmetric',
        eta='auto'
        #alpha='auto'  # Dirichlet-Hyperparameter Entscheidung zwischen Auto und symetric
    )
    
    id2word = dictionary
    dict_topic = {}
    # Themen anzeigen
    #print("Themen und ihre Schlüsselwörter:")
    for idx, topic in lda_model.print_topics(num_words=5):
        #print(f"Topic {idx}: {topic}")
        dict_topic[idx] = topic 

    return dict_topic


Stopwords: {'oh', 'you', 'to', 'ma', 'before', 'am', "shouldn't", 'herself', 'are', 'chorus', 'between', 'my', "you'd", "haven't", 'hook', 'wasn', 'because', 'have', 'how', 'like', 'further', 'outro', 'la', 'the', 'if', 'at', 'off', 'a', 're', 'don', 'this', 'having', 'shouldn', "shan't", 'in', 'he', 'very', 'just', 'won', 'your', 'been', 'after', 'our', 'more', 'there', 'than', 'too', 'weren', 'na', 'below', 'which', 'whom', 'few', "couldn't", 'about', 'doesn', 'once', 'couldn', "wasn't", "hasn't", 'be', 'she', "you'll", "wouldn't", 'verse', 'same', 'as', 'so', 'why', 'were', 'some', 'her', 'o', 'during', 'while', 'm', 'an', 'any', 'theirs', 'does', 'only', 'here', 'both', 'no', "aren't", 'of', "weren't", "needn't", 'himself', 'on', 'for', 'what', 't', 'with', "don't", 'hasn', 'again', 'yeah', 'ain', 'it', 'had', 'd', 'his', 'when', 'own', "you've", 'out', 'they', "doesn't", 'can', "she's", 'above', "mustn't", "mightn't", 'them', 'shan', 'against', 'not', 'was', 'over', 've', "won't",

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
amount = 1000
from datasets import load_from_disk

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"


rap_70_lyrics = process_subdatasets(load_from_disk(path + 'subdataset_70_pop')['lyrics'][:amount])
rap_80_lyrics = process_subdatasets(load_from_disk(path + "subdataset_80_pop")['lyrics'][:amount])
rap_90_lyrics = process_subdatasets(load_from_disk(path + "subdataset_90_pop")['lyrics'][:amount])
rap_00_lyrics = process_subdatasets(load_from_disk(path + "subdataset_00_pop")['lyrics'][:amount])
rap_10_lyrics = process_subdatasets(load_from_disk(path + "subdataset_10_pop")['lyrics'][:amount])
rap_20_lyrics = process_subdatasets(load_from_disk(path + "subdataset_20_pop")['lyrics'][:amount])

list_of_lyrics = {
    "70s": rap_70_lyrics,
    "80s": rap_80_lyrics,
    "90s": rap_90_lyrics,
    "00s": rap_00_lyrics,
    "10s": rap_10_lyrics,
    "20s": rap_20_lyrics
}

import csv

data_for_csv = []

for decade_name, decade in list_of_lyrics.items():
    i = 0
    for topics in decade:
        i = i + 1
        data_for_csv.append({
            "Method": "LDA",
            "Genre": "Pop",
            "Decade": decade_name,
            "Topic": str(decade[topics])
        })


# Datei schreiben
output_file = path + "Analysis_Folder/topics_pop_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


CSV-Datei '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/topics_pop_LDA.csv' wurde erfolgreich erstellt!


In [20]:
output_file = path + "Analysis_Folder/topics_pop_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

# tipps von ChaGPT
https://chatgpt.com/c/676f1ffc-b134-8012-9f03-6bd466bda148

- Herausfinden, wieviele Topics am geeingesten sind
- warum sind weiterhin so viele Stop words enthalten
- wieviele Lieder nehm ich überhaupt

In [ ]:
print(len(subdataset_80))

In [ ]:
print(len(subdataset_90))

<h1> BERT TOPIC MODELLING </h1>


In [179]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# clean the text
import re

def clean_text(document_list):
    
    cleaned_document_list = []
    
    for single_doc in document_list:
        # Delete HTML-Tags
        text = re.sub(r"<.*?>", "", single_doc)
        # Delete Sonderzeichen und Zahlen
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        # Delete uncessary whitespaces
        text = re.sub(r"\s+", " ", text).strip()
        
        # lower case the text
        text = text.lower()
        
        cleaned_document_list.append(text)
    
    return cleaned_document_list
    



# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess_stop_words(document_list):
    # Stopwords definieren
    stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "you","get", "the","and", "of", "on", "[Intro]" ,"[Storyteller]" ])
    
    list_of_tokens = []
    for single_doc in document_list:
        #without_strange_characters = re.sub(r"[^a-zA-Z0-9\s]", " ", single_doc) 
        #print("single_doc", single_doc)
        #print("single_doc_type", type(single_doc))
        #doc = remove_stopwords(doc.lower())
        #doc = remove_repeated_lines(doc)
        without_stop_words = [word for word in single_doc if word not in stop_words]
        print("without_stop_words", without_stop_words)
        print("type_without_stop_words", type(without_stop_words))
        
        
        tokens = word_tokenize(without_stop_words.lower())  
        print(tokens)
        
        list_of_tokens.append(tokens)
    
    return list_of_tokens

def remove_repeated_lines(doc):
    # doc ist eine Liste von Strings, z.B. ["Zeile1", "Zeile2", ...]
    # Wiederholte Zeilen entfernen und Reihenfolge beibehalten
    unique_lines = []
    seen = set()
    for line in doc:
        if line not in seen:
            unique_lines.append(line)
            seen.add(line)
    # Am Ende kannst du wieder eine Liste zurückgeben:
    return unique_lines

def sort_out_bad_words(doc,word_list, replacement=""):
    
    word_list = list(word_list) + ["like", "oh", "na", "la", "yo", "you","get", "the","and", "of", "on", "[Intro]" ,"[Storyteller]"]
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)

def tokenize_all_docs(document_list):
    list_of_tokens = []
    
    for single_doc in document_list:
        
        if type(single_doc) == str:
            tokens = word_tokenize(single_doc)
            list_of_tokens.append(tokens)
        else:
            print("Type von Document", type(single_doc))
            continue
    
    return list_of_tokens

print("Done")


Done


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [180]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import csv

def processBertTopics(dataset,sentenceTransformer):
    # Initialize your topic model
    #embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    embedding_model = SentenceTransformer(sentenceTransformer)
    topic_model = BERTopic(embedding_model=embedding_model)
    # Fit and transform the text data
    #without_repatition = remove_repeated_lines(dataset)
    
    cleaned_song_lyrics = clean_text(dataset)
    
    without_swear = [sort_out_bad_words(doc, swear_words ) for doc in cleaned_song_lyrics]
    
    #without_swear = preprocess(without_repatition)
    
    #print(type(without_swear))
    #print("without_swear",without_swear)
    
    
    # Semi supervised topuc modeling
    #classes = ["war", "love", "hate", "peace", "money", "family", "friends","hate"]
    #topics, probs = topic_model.topics_per_class(without_swear,classes)
    
    tokenized_docs = tokenize_all_docs(without_swear)
    
    topics, probs = topic_model.fit_transform(tokenized_docs)

    # Ergebnisse
    #print("Themenzuweisung:", topics)
    #print("Wahrscheinlichkeiten:", probs)

    str_topic_with_probs = []
    #for t, p in zip(topics, probs):
    #    str_topic_with_probs.append(str(t) + ": " + str(p))
        
    #return str_topic_with_probs
    #print(topic_model.get_topic_info())
    #print(type(topic_model.get_topics()))
    #print(topic_model.get_topics())
    #topic_model.visualize_topics()
    return topic_model.get_topics()
    
# Print or analyze the results
#print("Topics:", topics)
#print("Probabilities:", probs)
print("Done")


Done


In [ ]:
# import torch
# from transformers import AutoModel, AutoTokenizer

# # Modell und Tokenizer laden
# bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# # Anzahl der Songtexte, die verarbeitet werden sollen
# amount = 10  # Beispiel: Anzahl der Texte, die verarbeitet werden sollen

# # INPUT TWEET IST BEREITS NORMALISIERT!
# # Tokenisierung und Padding/Truncation
# lyrics = subdataset_90['lyrics'][:amount]
# inputs = tokenizer(lyrics, return_tensors="pt", padding=True, truncation=True, max_length=512)

# # Verarbeitung der Eingabe durch das Modell
# with torch.no_grad():
#     features = bertweet(**inputs)  # Models outputs sind jetzt Tupel

# # Ausgabe der Features (optional: erster Layer oder hidden states)
# print(features.last_hidden_state)  # Beispielausgabe
# print("Done")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


NameError: name 'subdataset_90' is not defined

In [181]:
import time
from datasets import load_from_disk
def run_analytics_bert_different_Sentence_Transformers(amount_per_iteration,name_of_run, sentence_transformer):

    start_time = time.time()
    amount = amount_per_iteration

    path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/pop/"

    
    rap_70_lyrics_bert = processBertTopics(load_from_disk(path + 'subdataset_70_pop')['lyrics'][:amount],sentence_transformer)
    rap_80_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_80_pop")['lyrics'][:amount],sentence_transformer)
    rap_90_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_90_pop")['lyrics'][:amount],sentence_transformer)
    rap_00_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_00_pop")['lyrics'][:amount],sentence_transformer)
    rap_10_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_10_pop")['lyrics'][:amount],sentence_transformer)
    rap_20_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_20_pop")['lyrics'][:amount],sentence_transformer)
    
    
    

    list_of_lyrics_bert = {
        "70s": rap_70_lyrics_bert,
        "80s": rap_80_lyrics_bert,
        "90s": rap_90_lyrics_bert,
        "00s": rap_00_lyrics_bert,
        "10s": rap_10_lyrics_bert,
        "20s": rap_20_lyrics_bert,
    }
    end_time = time.time()

    amount_of_time = end_time - start_time
    
    data_for_csv_bert = []
    
    for decade_name, decade in list_of_lyrics_bert.items():
        i = 0
        
        for topics in decade:
            i = i + 1
            data_for_csv_bert.append({
                "Method": "BERT",
                "Genre": "Rap",
                "Decade": decade_name,
                "Topic": str(decade[topics])
            })
    data_for_csv_bert.append({
                "Method": "TIME",
                "Genre": "TIME",
                "Decade": "TIME",
                "Topic": str(amount_of_time)
            })
    
    # Write to csv
    path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/BERT/"
   # Datei schreiben
    name_of_file = "topics_pop_BERT_" + "MiniLM.csv"
    output_file = os.path.join(path, name_of_file)

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
        writer.writeheader()  
        writer.writerows(data_for_csv_bert) 
   
print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")
print("Done")


CSV-Datei '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/BERT/topics_rap_BERT_MiniLM.csv' wurde erfolgreich erstellt!
Done


In [182]:
run_analytics_bert_different_Sentence_Transformers(1000,"TEST_MIST","all-MiniLM-L6-v2")

TypeError: Make sure that the iterable only contains strings.